In [35]:
def grep(pattern):
    print("start grep for", pattern)
    count = 0

    while True:
        s = yield count
            
        count += 1
        if pattern in s:
            print("found!", s)
        else:
            print("no %s in %s" % (pattern, s))

In [26]:
g = grep("python")
g

<generator object grep at 0x103fd1d20>

In [27]:
next(g)

start grep for python


0

In [28]:
g.send("None")
g.send("python")

no python in None
found! python


2

In [33]:
g.send("python")

found! python


7

In [34]:
g.throw(ValueError("WRONG"))

ValueError: WRONG

In [39]:
import asyncio

In [42]:
async def countdown(n):
    print(f"start count {n}")
    for i in range(n):
        print(f"count {i + 1} of {n}")
        await asyncio.sleep(1)

In [43]:
await countdown(3)

start count 3
count 1 of 3
count 2 of 3
count 3 of 3


In [44]:
c = countdown(3)

In [45]:
del c

/var/folders/c1/9wdb7dps2x332c6l1y2hrqkh0000gp/T/ipykernel_9145/1625600815.py:1: RuntimeWarning: coroutine 'countdown' was never awaited
  del c


In [48]:
import time

In [49]:
async def add_counts():
    await countdown(2)
    await countdown(3)


t1 = time.time()

await add_counts()

t2 = time.time()
print("time", t2 - t1)

start count 2
count 1 of 2
count 2 of 2
start count 3
count 1 of 3
count 2 of 3
count 3 of 3
time 5.004453897476196


In [50]:
async def add_counts():
    c = countdown(2)
    await countdown(3)
    await c


t1 = time.time()

await add_counts()

t2 = time.time()
print("time", t2 - t1)

start count 3
count 1 of 3
count 2 of 3
count 3 of 3
start count 2
count 1 of 2
count 2 of 2
time 5.003523111343384


In [51]:
async def add_counts():
    task = asyncio.create_task(countdown(2))
    await task
    await countdown(3)


t1 = time.time()

await add_counts()

t2 = time.time()
print("time", t2 - t1)

start count 2
count 1 of 2
count 2 of 2
start count 3
count 1 of 3
count 2 of 3
count 3 of 3
time 5.014102935791016


In [52]:
async def add_counts():
    task = asyncio.create_task(countdown(2))
    await countdown(3)
    await task


t1 = time.time()

await add_counts()

t2 = time.time()
print("time", t2 - t1)

start count 3
count 1 of 3
start count 2
count 1 of 2
count 2 of 3
count 2 of 2
count 3 of 3
time 3.003131151199341


In [55]:
async def block(n):
    for i in range(n):
        print(f"block {i + 1} of {n}")
        time.sleep(1)

In [56]:
await block(3)

block 1 of 3
block 2 of 3
block 3 of 3


In [58]:
async def add_counts():
    task = asyncio.create_task(countdown(2))
    await block(3)
    await countdown(3)
    await task


t1 = time.time()

await add_counts()

t2 = time.time()
print("time", t2 - t1)

block 1 of 3
block 2 of 3
block 3 of 3
start count 3
count 1 of 3
start count 2
count 1 of 2
count 2 of 3
count 2 of 2
count 3 of 3
time 6.006447076797485


In [61]:
async def block(n):
    for i in range(n):
        print(f"block {i + 1} of {n}")
        time.sleep(1)
        await asyncio.sleep(0)


async def add_counts():
    task = asyncio.create_task(countdown(2))
    await block(3)
    await countdown(3)
    await task


t1 = time.time()

await add_counts()

t2 = time.time()
print("time", t2 - t1)

block 1 of 3
start count 2
count 1 of 2
block 2 of 3
block 3 of 3
start count 3
count 1 of 3
count 2 of 2
count 2 of 3
count 3 of 3
time 6.014416933059692


In [63]:
!pip install aiohttp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.1/358.1 kB 1.7 MB/s eta 0:00:00a 0:00:01
  Using cached frozenlist-1.3.3-cp310-cp310-macosx_10_9_x86_64.whl (35 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.8/124.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.9 MB/s eta 0:00:00
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip


In [64]:
import aiohttp

In [71]:
URL = "https://docs.python.org/3/whatsnew/3.11.html"
URLS = [URL] * 50


async def fetch_url(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as resp:
            assert resp.status == 200
            

async def fetch_batch(urls):
    for url in urls:
        await fetch_url(url)
        
t1 = time.time()

await fetch_batch(URLS)

t2 = time.time()
print("TIME", t2 - t1)

TIME 9.064406871795654


In [78]:
URL = "https://docs.python.org/3/whatsnew/3.11.html"
URLS = [URL] * 100
count = 0


async def fetch_url(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as resp:
            global count
            count += 1
            assert resp.status == 200
            

async def fetch_batch(urls):
    tasks = [
        asyncio.create_task(fetch_url(url))
        for url in urls
    ]
    await asyncio.gather(*tasks)

        
t1 = time.time()

await fetch_batch(URLS)

t2 = time.time()
print("TIME", t2 - t1, count)

TIME 0.373183012008667 100


In [84]:
URL = "https://docs.python.org/3/whatsnew/3.11.html"
URLS = [URL] * 100
count = 0


async def fetch_url(url, sem):
    async with aiohttp.ClientSession() as session:
        async with sem:
            async with session.get(url) as resp:
                global count
                count += 1
                assert resp.status == 200


async def fetch_batch(urls, sem):
    tasks = [
        asyncio.create_task(fetch_url(url, sem))
        for url in urls
    ]
    await asyncio.gather(*tasks)


t1 = time.time()

sem = asyncio.Semaphore(10)
await fetch_batch(URLS, sem)

t2 = time.time()
print("TIME", t2 - t1, count)

TIME 1.8035788536071777 100


In [85]:
t1 = time.time()

asyncio.run(fetch_batch(URLS, sem))

t2 = time.time()
print("TIME", t2 - t1, count)

RuntimeError: asyncio.run() cannot be called from a running event loop

In [91]:
URL = "https://docs.python.org/3/whatsnew/3.11.html"
URLS = [URL] * 100
WORKERS = 10
count = 0


async def fetch_url(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as resp:
            global count
            count += 1
            assert resp.status == 200
            
            
async def run_worker(queue):
    while True:
        url = await queue.get()
        try:
            await fetch_url(url)
        finally:
            queue.task_done()


async def fetch_batch(urls):
    queue = asyncio.Queue()
    
    workers = [
        asyncio.create_task(run_worker(queue))
        for _ in range(WORKERS)
    ]
    
    for url in urls:
        await queue.put(url)

    await queue.join()

    for worker in workers:
        worker.cancel()


t1 = time.time()

await fetch_batch(URLS)

t2 = time.time()
print("TIME", t2 - t1, count)

TIME 1.8839318752288818 100
